In [1]:
#This notebook uses an API that was built utilizing the ESPN Fantasy Football API 
#The documentation can be found here:
#https://github.com/cwendt94/ff-espn-api

%matplotlib inline
#The necessary packages
import pandas as pd
import numpy as np
import seaborn as sns
from ff_espn_api import League

#Info:
league_id=4570371
season=2019
#team='10'

In [101]:
#This is the personal league information
league=League(league_id,2019)
myTeam=league.teams[9]

In [184]:
#Historical data: all credit goes to this page:
#https://github.com/derek-adair/nflgame
#The documentation for all of the functionality here:
#http://nflgame.derekadair.com/
#However, the more idiot proof guide is here (bless this person!!)
#https://github.com/BurntSushi/nflgame/wiki/Tutorial-for-non-programmers:-Installation-and-examples
import nflgame

#Games from 2018
games=nflgame.games(2018)
players=nflgame.combine_play_stats(games)
player_list=[p for p in players]
player_names=[p.name for p in players]
#Note: a guide to all available player stats can be found here: https://github.com/BurntSushi/nflgame/wiki/Stat-types

In [185]:
#At the moment, the draft hasn't happened yet. 
#Filter the free agents by position and ranking
positions=['QB','RB','WR','TE','FLEX','K']
positionDict={}
for pos in positions:
    playerList=league.free_agents(position=pos)
    playerDict={}
    for p in playerList:
        playerDict[p.name]={'Ranking':p.posRank,'Team':p.proTeam}
    positionDict[pos]=playerDict

#We'll keep a separate tally for defense 
defenseList=league.free_agents(position='D/ST')
defenseDict={}
for d in defenseList:
    defenseDict[d.name]={'Ranking':d.posRank,'Team':d.proTeam}

In [190]:
#Can we add more detailed stats to the list of players?
for pos in positions:
    sub_dict=positionDict[pos]
    for p in sub_dict.keys():
        finit=p[0]
        lastname=p.split(" ")[1]
        nameFormat='{:}.{:}'.format(finit,lastname)
        #Check if player is in list
        pcheck=(nameFormat in player_names)
        if pcheck==True:
            pIndex=player_names.index(nameFormat)
            playerStats=player_list[pIndex].stats
            sub_dict[p]['stats']=playerStats
    positionDict[pos]=sub_dict




In [214]:
#If you want it in dataframe format...
playerFrame=pd.DataFrame.from_dict({(i,j): positionDict[i][j] 
                           for i in positionDict.keys() 
                           for j in positionDict[i].keys()},
                       orient='index')
playerFrame=playerFrame.reset_index()
playerFrame.columns=['Position','Name','Ranking','Team','Stats']
playerFrame=playerFrame.dropna(subset=['Stats'])

In [216]:
fullFrame=pd.concat([playerFrame.drop(['Stats'], axis=1), playerFrame['Stats'].apply(pd.Series)], axis=1)
fullFrame.to_csv('2')

In [101]:
#Another version:
#Get the JSON table in easily readable format
#Basic team info
url='https://fantasy.espn.com/apis/v3/games/ffl/seasons/2019/segments/0/leagues/{:}'.format(league_id)
r=requests.get(url)
dat=r.json()

In [ ]:
#Get the weekly matchup scores
slotcodes={0:'QB',2:'RB',4:'WR',
          6:'TE',16:'Def',17:'K',
          20:'Bench',21:'IR',23:'Flex'
          }
url = 'https://fantasy.espn.com/apis/v3/games/ffl/seasons/' + \
      str(season) + '/segments/0/leagues/' + str(league_id) + \
      '?view=mMatchup&view=mMatchupScore'


In [ ]:
data=[]
print('Week ',end='')
for week in range(1,2):
    print(week,end=' ')
    r=requests.get(url,params={'scoringPeriodId':week})
    d=r.json()
    for tm in d['teams']:
        tmid=tm['id']
        for p in tm['roster']['entries']:
            name=p['playerPoolEntry']['player']['fullName']
            slot=p['lineupSlotId']
            pos=slotcodes[slot]
            
            inj='NA'
            try:
                inj=p['playerPoolEntry']['player']['injuryStatus']
            except:
                pass
        proj,act=None,None
        for stat in p['playerPoolEntry']['player']['stats']:
            if stat['scoringPeriodId']!=week:
                continue
            if stat['statSourceId'] == 0:
                act = stat['appliedTotal']
            elif stat['statSourceId']==1:
                proj=stat['appliedTotal']
        data.append([
                week, tmid, name, slot, pos, inj, proj, act
        ])

In [ ]:
data = pd.DataFrame(data, 
                    columns=['Week', 'Team', 'Player', 'Slot', 
                             'Pos', 'Status', 'Proj', 'Actual'])